# ParlAI "tutorial"

In [1]:
from parlai.core.params import ParlaiParser
from parlai.agents.repeat_label.repeat_label import RepeatLabelAgent
from parlai.core.worlds import create_task

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
args = ["-t", "convai2"]
parser = ParlaiParser()
parser.add_argument("-n", "--num-examples", default=5, type=int)
opt = parser.parse_args(args)  # special parlai Opt class, seem to be like a dict

In [4]:
agent = RepeatLabelAgent(opt)

# Worlds
* https://parl.ai/docs/tutorial_worlds.html
* https://github.com/facebookresearch/ParlAI/blob/master/parlai/core/worlds.py

world.parley():   Agent 0 goes first.  Alternate between the two agents.

BathWorlds:

pseudocode in DialogPartnerWorld

```
query = teacher.act()
student.observe(query)
reply = student.act()
teacher.observe(reply)
```

In [5]:
world = create_task(opt, agent)  # DialogPartnerWorld
for i, agent in enumerate(world.agents):
    print(f"Agent {i}:", agent.__class__.__name__)

17:09:55 | creating task(s): convai2
17:09:55 | loading fbdialog data: /home/erik/miniconda3/envs/ttd/lib/python3.8/site-packages/data/ConvAI2/train_self_original.txt
Agent 0: DefaultTeacher
Agent 1: RepeatLabelAgent


In [6]:
for i in range(opt["num_examples"]):
    world.parley()
    print(world.display() + "\n")
    if world.epoch_done():
        print("epoch done")
        break

[convai2]: your persona: i want to be in a band someday.
your persona: i've a wife and two kids.
your persona: my hobbies are sleeping and playing the guitar.
your persona: i sleep 10 hours every day because my work is tiring.
your persona: i am a factory worker.
__SILENCE__
[labels]: hi , how are you today
   [RepeatLabelAgent]: hi , how are you today

[convai2]: i am fine , how are you today ?
[labels]: i need a nap , i love sleeping it is a hobby !
   [RepeatLabelAgent]: i need a nap , i love sleeping it is a hobby !

[convai2]: i love to nap , i need it after my long shifts . what do you do for work ?
[labels]: i'm tried from work , i work in a factory so that is hard work .
   [RepeatLabelAgent]: i'm tried from work , i work in a factory so that is hard work .

[convai2]: i work as a paramedic , so we get a lot of stress on the job .
[labels]: but one day i will be in a rock band , that's my dream !
   [RepeatLabelAgent]: but one day i will be in a rock band , that's my dream !

[

## Custom Agent

We follow the example given in 

https://parl.ai/docs/tutorial_torch_generator_agent.html#tutorial

In [7]:
# TorchGeneratorAgent
import parlai.core.torch_generator_agent as tga
import torch
import torch.nn as nn
import torch.nn.functional as F

### Model

Extends tga.TorchGeneratorModel which is simply a wrapper/extension of the regular nn.Module class from pytorch

In [8]:
class GenModel(tga.TorchGeneratorModel):
    def __init__(self, dictionary, hidden_size=256):
        super().__init__(
            padding_idx=dictionary["padding_idx"],
            start_idx=dictionary["start_idx"],
            end_idx=dictionary["end_idx"],
            unknown_idx=dictionary["unknown_idx"],
        )
        self.embeddings = nn.Embedding(len(dictionary), hidden_size)
        self.encoder = Encoder(self.embeddings, hidden_size)
        self.decoder = Decoder(self.embeddings, hidden_size)

    def output(self, decoder_output):
        return F.linear(decoder_output, self.embeddings.weight)

    def reorder_encoder_states(self, encoder_states, indices):
        h, c = encoder_states
        return h[:, indices, :], c[:, indices, :]

    def reorder_decoder_incremental_state(self, incr_state, indices):
        h, c = incr_state
        return h[:, indices, :], c[:, indices, :]


class Encoder(nn.Module):
    def __init__(self, embeddings, hidden_size):
        super().__init__()
        _vocab_size, esz = embeddings.weight.shape
        self.embeddings = embeddings
        self.lstm = nn.LSTM(
            input_size=esz, hidden_size=hidden_size, num_layers=1, batch_first=True
        )

    def forward(self, input_tokens):
        embedded = self.embeddings(input_tokens)
        _output, hidden = self.lstm(embedded)
        return hidden


class Decoder(nn.Module):
    def __init__(self, embeddings, hidden_size):
        super().__init__()
        _vocab_size, self.esz = embeddings.weight.shape
        self.embeddings = embeddings
        self.lstm = nn.LSTM(
            input_size=self.esz, hidden_size=hidden_size, num_layers=1, batch_first=True
        )

    def forward(self, input, encoder_state, incr_state=None):
        embedded = self.embeddings(input)
        if incr_state is None:
            state = encoder_state
        else:
            state = incr_state
        output, incr_state = self.lstm(embedded, state)
        return output, incr_state

### Agent

Now we extend the class tga.TorchGeneratorAgent in a way such that it uses our PyTorch model.

In [10]:
class CustomGenAgent(tga.TorchGeneratorAgent):
    @classmethod
    def add_cmdline_args(cls, argparser):
        super(CustomGenAgent, cls).add_cmdline_args(argparser)
        group = argparser.add_argument_group("Example TGA Agent")
        group.add_argument(
            "-hid", "--hidden-size", type=int, default=256, help="Hidden size."
        )

    def build_model(self):
        model = GenModel(self.dict, self.opt["hidden_size"])
        if self.opt["embedding_type"] != "random":
            self._copy_embeddings(model.embeddings.weight, self.opt["embedding_type"])
        return model

Lets try to use our custom agent instead of the simple `RepeatLabelAgent`.

In [14]:
# Get the opts
args = ["-t", "convai2"]
# args = ["-t", "dailydialog"]
parser = ParlaiParser()
CustomGenAgent.add_cmdline_args(parser)
parser.add_argument("-n", "--num-examples", default=5, type=int)
opt = parser.parse_args(args)  # special parlai Opt class, seem to be like a dict


# Create agent
agent = CustomGenAgent(opt)

# Show the agents of the world and task
world = create_task(opt, agent)  # DialogPartnerWorld
print()
for i, agent in enumerate(world.agents):
    print(f"Agent {i}:", agent.__class__.__name__)
    if hasattr(agent, "model"):
        print('='*50)
        print("model: ", agent.model)
        print('='*50)

17:17:55 | Using CUDA
17:17:55 | Total parameters: 1,053,696 (1,053,696 trainable)
17:17:55 | creating task(s): convai2
17:17:55 | loading fbdialog data: /home/erik/miniconda3/envs/ttd/lib/python3.8/site-packages/data/ConvAI2/train_self_original.txt

Agent 0: DefaultTeacher
Agent 1: CustomGenAgent
model:  GenModel(
  (embeddings): Embedding(4, 256)
  (encoder): Encoder(
    (embeddings): Embedding(4, 256)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embeddings): Embedding(4, 256)
    (lstm): LSTM(256, 256, batch_first=True)
  )
)


Lets run world.parley() again to see how it works

In [17]:
print("agent.observation: BEFORE", agent.observation)
try:
    for i in range(opt["num_examples"]):
        world.parley()
        print(world.display() + "\n")
        if world.epoch_done():
            print("epoch done")
            break
except:
    pass
            
print("\n\nagent.observation AFTER: ", agent.observation)

agent.observation: BEFORE {'text': 'your persona: my favorite thing to do is float the river.\nyour persona: my favorite activity is fishing.\nyour persona: i once ran the mile in under 4 minutes.\nyour persona: i cannot go a single day without eating ice cream.\nhi ! i am jovanni . whats up ? tell me about yourself .', 'labels': ['i wo not be doing any floating in the river this weekend . darn irma .'], 'reward': 0, 'label_candidates': ['hi ! good because i played hooky today . . . just baking my fave brownies today . you ?', "that's not fun , i hope things work out", 'are you the lady i gave my number to last night ?', 'thank you , i feel the same way', 'no bacon for me but i love chocolate milkshakes', 'yes i have both red and black', 'oh right yes . probably the same from batman .', 'i go to spas and hang out with my dog .', 'sounds like a lot of fun !', 'yes . i can make anyone look pretty', 'i sing opera , and boil pasta . my dog is my only friend . what can we do ?', 'do you run

### Assertion Error
The model checks that the first tokens are correct (not multiple bos-tokens) and it fails. Throwing this error:

```
AssertionError: The Beginning of Sentence token is automatically added to the label in decode_forced, but you included it in the label. This means your model will have a double BOS token, which is probably not what you intended.
```

We can see from our agents observations that all text got mapped onto the same symbol (probably unk-token??). So the tokenization is not handled correct for our model.

In [19]:
world.reset()

### Tokenizer

ParlAI does a good job abstracting a way the notion of a tokenizer. It is somewhat difficult to find where and how the tokenizer is used but you'll soon end up on `DictionaryAgent`.


The DictionaryAgent handles the process of turning text into vectors and back... Our lookup table / vocab. In my work I want to be able to use pretrained tokenizers from the huggingface framwork but also remove capitalization and punctuation, making the text more simliar to that of spoken dialog interaction, and so we are going to implement our own DictionaryAgent based on information found in the ParlAI transformer code.

Im going to use a normalizer from the HuggingFace Tokenizer library (fast tokenization written in rust) to remove punctuation and capitalization but then feed that text into some pretrained tokenizer (i.e. Bert, GPT2).

In [21]:
# Custom Tokenizer
from parlai.core.dict import DictionaryAgent
from transformers import AutoTokenizer
from parlai.utils.misc import warn_once
from tokenizers import Tokenizer
from tokenizers import pre_tokenizers, normalizers, decoders, Regex
from tokenizers.normalizers import Lowercase, NFD, StripAccents, Replace, Strip

In [22]:
class SpokenDialogDictionaryAgent(DictionaryAgent):
    """
    Allow to use the Torch Agent with the wordpiece dictionary of Hugging Face.
    """

    @classmethod
    def add_cmdline_args(cls, argparser):
        argparser.add_argument("--tokenizer", type=str, default="gpt2")

    @staticmethod
    def build_normalizer():
        normalizer = normalizers.Sequence(
            [
                NFD(),
                Lowercase(),
                StripAccents(),
                Replace(Regex('[\.\,\!\?\:\;\)\(\[\]"\-]'), ""),
                Strip(),
            ]
        )
        return normalizer

    def __init__(self, opt):
        # initialize from vocab path
        warn_once(
            "WARNING: TurnTaking uses a TurnTaking tokenizer; ParlAI dictionary args are ignored"
        )
        # download(opt["datapath"])
        self._tokenizer = AutoTokenizer.from_pretrained(opt["tokenizer"])
        self.normalizer = SpokenDialogDictionaryAgent.build_normalizer()
        super().__init__(opt)

        if opt["tokenizer"].startswith("gpt2"):
            self.start_token = "[CLS]"
            self.cls_token = "[CLS]"
            self.end_token = "[SEP]"
            self.pad_token = "[PAD]"
            self.null_token = "[PAD]"
            self.mask_token = "[MASK]"
            self.unk_token = "[UNK]"
        elif opt["tokenizer"].startswith("bert"):
            self.start_token = "[bos"
            self.end_token = "[SEP]"
            self.pad_token = "<end-of-text>"
            self.null_token = "<end-of-text>"
            self.mask_token = "[MASK]"
            self.unk_token = "[UNK]"

        self.start_idx = self._tokenizer.convert_tokens_to_ids(self.start_token)
        self.end_idx = self._tokenizer.convert_tokens_to_ids(self.end_token)
        self.pad_idx = self._tokenizer.convert_tokens_to_ids(self.pad_token)
        self.unk_idx = self._tokenizer.convert_tokens_to_ids(self.unk_token)
        self.null_idx = self._tokenizer.convert_tokens_to_ids(self.null_token)

        self.update_vocab()

    def update_vocab(self):
        # set tok2ind for special tokens
        self.tok2ind[self.start_token] = self.start_idx
        self.tok2ind[self.end_token] = self.end_idx
        self.tok2ind[self.pad_token] = self.pad_idx
        self.tok2ind[self.unk_token] = self.unk_idx

        # set ind2tok for special tokens
        self.ind2tok[self.start_idx] = self.start_token
        self.ind2tok[self.end_idx] = self.end_token
        self.ind2tok[self.pad_idx] = self.null_token
        self.ind2tok[self.unk_idx] = self.unk_token

    def __len__(self):
        return len(self._tokenizer)

    def encode(self, text, add_special_tokens=False):
        text = self.normalizer.normalize_str(text)
        return self._tokenizer.encode(text, add_special_tokens=add_special_tokens)

    def txt2vec(self, text, vec_type=list):
        return self.encode(text)

    def vec2txt(self, vec):
        if not isinstance(vec, list):
            # assume tensor
            vec = vec.tolist()
        return self._tokenizer.decode(vec)

    def act(self):
        return {}

#### Update our Agent class

Now we also must update our agent class to tell it which DictionaryAgent it should rely on. This is easily done by overwriting a function in the agent class namely `dictionary_class()`.

In [23]:
class CustomGenAgent(tga.TorchGeneratorAgent):
    @classmethod
    def add_cmdline_args(cls, argparser):
        argparser.set_defaults(dict_maxexs=0)  # skip building dictionary)
        super(CustomGenAgent, cls).add_cmdline_args(argparser)
        cls.dictionary_class().add_cmdline_args(argparser)
        group = argparser.add_argument_group("Example TGA Agent")
        group.add_argument(
            "-hid", "--hidden-size", type=int, default=256, help="Hidden size."
        )

    @staticmethod
    def dictionary_class():
        """
        Return the dictionary class that this agent expects to use.

        Can be overriden if a more complex dictionary is required.
        """
        return SpokenDialogDictionaryAgent

    def build_model(self):
        model = GenModel(self.dict, self.opt["hidden_size"])
        if self.opt["embedding_type"] != "random":
            self._copy_embeddings(model.embeddings.weight, self.opt["embedding_type"])
        return model

In [25]:
# Create agent
parser = ParlaiParser()
CustomGenAgent.add_cmdline_args(parser)
parser.add_argument("-n", "--num-examples", default=5, type=int)
opt = parser.parse_args(args)  # special parlai Opt class, seem to be like a dict
print('tokenizer: ', opt['tokenizer'])

tokenizer:  gpt2


In [27]:
agent = CustomGenAgent(opt)
world = create_task(opt, agent)  # DialogPartnerWorld

# Show the agents of the world and task
print()
for i, agent in enumerate(world.agents):
    print(f"Agent {i}:", agent.__class__.__name__)
    if hasattr(agent, "model"):
        print('='*50)
        print("model: ", agent.model)
        print('='*50)
        
print()
print("agent.observation: ", agent.observation)
for i in range(opt["num_examples"]):
    world.parley()
    print(world.display() + "\n")
    if world.epoch_done():
        print("epoch done")
        break
print("agent.observation: ", agent.observation)

17:38:23 | Using CUDA
17:38:25 | Total parameters: 13,918,464 (13,918,464 trainable)
17:38:25 | creating task(s): convai2
17:38:25 | loading fbdialog data: /home/erik/miniconda3/envs/ttd/lib/python3.8/site-packages/data/ConvAI2/train_self_original.txt

Agent 0: DefaultTeacher
Agent 1: CustomGenAgent
model:  GenModel(
  (embeddings): Embedding(50257, 256)
  (encoder): Encoder(
    (embeddings): Embedding(50257, 256)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embeddings): Embedding(50257, 256)
    (lstm): LSTM(256, 256, batch_first=True)
  )
)

agent.observation:  None
[convai2]: your persona: i have never broken a bone.
your persona: i love cats.
your persona: i am a freshman in college.
your persona: i am athletic.
your persona: my favorite food is pizza.
hello , how are you doing today ?
[labels]: i am good especially because my cat is cuddling with me ! you ?

[convai2]: i wish i had a cat . i only have children haha .
[labels]: cool , its my first ye

### It Works!

So now the agent uses a correctly defined DictionaryAgent and we can see that the text have been encoded correctly. The DictionaryAgent can be accessed by `agent.dict`. Lets see how the text is changed between encoding and decoding (because of our custom normalizer).


So we are now a step closer in being able to train our own model. 

In [30]:
print(agent.observation['text'])
vec = agent.dict.txt2vec(agent.observation['text'])
print('vec: ', vec)
txt = agent.dict.vec2txt(vec)
print('decoded text: ', txt)

i would love to go some day !
vec:  [72, 561, 1842, 284, 467, 617, 1110]
decoded text:  i would love to go some day


As we can see all the words gets remapped correctly except the exclamation point (as desired)